In [ ]:
from torch.quantization import MovingAverageMinMaxObserver
from torch.ao.quantization.observer import MinMaxObserver
from torch.quantization import QuantStub, DeQuantStub
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
import torch.optim as optim
import numpy as np
from torchvision import datasets
from art.estimators.classification import PyTorchClassifier
from art.utils import load_cifar10
from art.preprocessing.standardisation_mean_std import StandardisationMeanStdPyTorch
from art.attacks.evasion.hop_skip_jump import HopSkipJump
import time
import pickle
import os
from math import log10, sqrt
import cv2
from torch.utils.data import DataLoader
torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

def print_size_of_model(model):
    """ Prints the real size of the model """
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

    
def accuracy(output, target):
    """ Computes the top 1 accuracy """
    with torch.no_grad():
        batch_size = target.size(0)

        _, pred = output.topk(1, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        correct_one = correct[:1].view(-1).float().sum(0, keepdim=True)
        return correct_one.mul_(100.0 / batch_size).item()

def predict(img):
  img = torch.from_numpy(img)
  img = F.normalize(img, [0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
  return img.numpy()


def test(model: nn.Module, dataloader: DataLoader, cuda=False) -> float:
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            if cuda:
              inputs = inputs.cuda()
              labels = labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [ ]:
# Step 1: Load the CIFAR10 dataset

(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_cifar10()

# Step 1a: Swap axes to PyTorch's NCHW format

x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_data = datasets.CIFAR10(root="./data",train=False,download=True,transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

# show images
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [ ]:
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=True)
model.to("cuda")
model.eval()
model.float()


# Step 2a: Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, dampening=0,weight_decay=0.0005, nesterov=True)


score = test(model, test_loader, cuda=True)
print('Accuracy of the network on the test images: {}% - FP32'.format(score))
print("Size of model before quantization")
print_size_of_model(model)

# Step 3: Create the ART classifier
classifier = PyTorchClassifier(
    model=model,
    clip_values=(0.0, 1.0),
    loss=criterion,
    optimizer=optimizer,
    preprocessing=((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    input_shape=(3, 32, 32),
    nb_classes=10,
    device_type = "gpu"
)
print("HopSkipJump Attack Initialization")

attack = HopSkipJump(classifier,64,targeted = False,verbose = True)

for i in range(0,100):
    x_train_hop_res20_cifar = attack.generate(predict(x_train[0+300*i:300*(i+1)]),y_train[0+300*i:300*(i+1)])
    with open("x_train_hop_res20_cifar_{}_to_{}.pkl".format(300*i,300*(i+1)),'wb') as f:
        pickle.dump(x_train_hop_res20_cifar,f)
    with open("resnet20_cifar_progress.txt",'wb') as f:
        f.write("Done with {}\n".format(300*(i+1)))